In [ ]:
# для того, чтобы увеличить максииальный объём файла, который можно загружать в память
import sys
import csv
def find_and_set_sys_maxsize():
    maxInt = sys.maxsize
    decrement = True
    while decrement:
        decrement = False
        try:
            csv.field_size_limit(maxInt)
        except OverflowError:
            maxInt = int(maxInt/10)
            decrement = True
find_and_set_sys_maxsize()

# отключим ворнинги
import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

import os
import pandas as pd
import numpy as np

import gensim

# штука для предобработки текста
import nltk
nltk.download('punkt')
# from nltk.tokenize import word_tokenize

import pickle
import math
import datetime
def timestamp(): return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from scipy.sparse import hstack

import xgboost as xgb

### Read the Data preparation _ Baseline file for some data preprocessing explanations

In [2]:
# Print iterations progress
def print_progress_bar(iteration, total, prefix = '', suffix = '', decimals = 0):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    length = 50
    fill = '='
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    progress_bar = fill * filled_length + '-' * (length - filled_length)
    print('\r%s [%s] %s%% %s' % (prefix, progress_bar, percent, suffix), end='\r')
    # Print new line on complete
    if iteration == total:
        print()

In [3]:
granddebat_folder = '../GrandDébat/granddebat.fr/'
democrat_path = os.path.join(granddebat_folder, 'DEMOCRATIE_ET_CITOYENNETE.csv')
events_path = os.path.join(granddebat_folder, 'EVENTS.csv')
fiscalit_path = os.path.join(granddebat_folder, 'LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.csv')
ecolog_path = os.path.join(granddebat_folder, 'LA_TRANSITION_ECOLOGIQUE.csv')
organisat_path = os.path.join(granddebat_folder, 'ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.csv')

french_encoding = "utf-8"

In [4]:
file_path = democrat_path

In [5]:
db_raw = pd.read_csv(file_path, engine='python', encoding=french_encoding)

In [6]:
questions_reverse_index = 37
db_questions = db_raw.loc[:, db_raw.columns[-questions_reverse_index:]]

question_list = db_raw.columns[-questions_reverse_index:]
question_list = [el.split()[0][-2:] for el in question_list]
db_questions.columns = question_list

db_questions.head()

,A3,A4,A5,Ew,Ex,Ey,Ez,E0,E1,E2,...,M1,M2,M3,M4,M5,Qx,Qy,Qz,Q0,Q1
0,Le citoyen,Non,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afin d’éviter de creuser les inégalités ne plu...
1,Un instrument de démocratie locale à modernise...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nous proposons le retour à la limitation de vi...
3,Voir l'intégralité de la proposition dans la d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POUR UN NOUVEAU CONTRAT CITOYEN ...
4,"député, maire, moi même",Non,NaN,"Budget participatif, possibilité d'interpeller...",Une bonne chose,NaN,NaN,Oui,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Accumulated by postal regions (100 of them)

In [6]:
def preprocess_text(text):
    text = " ".join(gensim.utils.simple_preprocess(text))
    return text

def preprocess_zipCode(zipCode):
    return int(str(zipCode)[0:2])

Here we get the postal region and answer text out of the dataframe, taking only first two digits as meaningful

In [7]:
db = db_questions

texts = []
authorZipCode_regions = []
question_numbers = []
i = 0

for row in range(db.shape[0]):
    if row % 10000 == 0: print("row", row, "/", db.shape[0])
    for col in range(db.shape[1]):
        if not pd.isnull(db.iat[row, col]):
            text = preprocess_text(db.iat[row, col])
            authorZipCode_region = preprocess_zipCode(db_raw.at[row, 'authorZipCode'])
            question_number = col

            texts.append(text)
            authorZipCode_regions.append(authorZipCode_region)
            question_numbers.append(question_number)

row 0 / 65758
row 10000 / 65758
row 20000 / 65758
row 30000 / 65758
row 40000 / 65758
row 50000 / 65758
row 60000 / 65758


### TF-IDF
This is a pretty powerful thus non computationally expensive method of analyzing texts  
It counts the number of word occurences in the documents  
Based on this, we can guess how important are some words for the document (how frequent they are in the document divided by how frequent they are in all the documents)  

train_tfidf_texts is a sparse matrix that has mostly zeros  
rows represent documets (answers here) and columns words in the vocabulary formed by the tf-idf  

In [8]:
vectorizer = TfidfVectorizer(min_df=1)
train_tfidf_texts = vectorizer.fit_transform(texts)

In [37]:
# X = train_tfidf_texts
# adding information about the question number increases accurcy very little
X = hstack((train_tfidf_texts, np.array(question_numbers)[:,None]))
y = authorZipCode_regions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

### XGBoost

In [38]:
dtrain = xgb.DMatrix(X_train, label = y_train)

In [39]:
param = {}
# param['silent'] = 1
# param['verbosity'] = 2
param['nthread'] = 8

param['max_depth'] = 6
num_boost_round = 2


bst = xgb.train(param, dtrain, num_boost_round = num_boost_round)

[11:51:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=6
[11:51:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 78 extra nodes, 0 pruned nodes, max_depth=6


In [40]:
xgb_X_test = xgb.DMatrix(X_test)
xgb_y_test = xgb.DMatrix(y_test)

y_pred = bst.predict(xgb_X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Model accuracy: %.2f%%" % (accuracy * 100.0))
random_accuracy = 1 / len(set(authorZipCode_regions))
print("Random accuracy: %.2f%%" % (random_accuracy * 100.0))

Model accuracy: 1.40%
Random accuracy: 1.02%


### Naive Bayes

In [43]:
from sklearn.naive_bayes import MultinomialNB as NB

# fit_prior = False decreases acc by 0.1% which means that we have a slight imbalance between zipcodes
nb_model = NB().fit(X=X_train, y=y_train)

In [44]:
y_pred = nb_model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Model accuracy: %.2f%%" % (accuracy * 100.0))
random_accuracy = 1 / len(set(authorZipCode_regions))
print("Random accuracy: %.2f%%" % (random_accuracy * 100.0))

Model accuracy: 6.40%
Random accuracy: 1.02%


## Accumulated by regions (? of them)

In [7]:
region_names_to_zipCode_regions = {
"Île de France": ["75", "77", "78", "91", "92", "93", "94", "95"],
"Nord-Pas-de-Calais Picardie": ["02", "59", "60", "62", "80"],
"Alsace Champagne-Ardenne Lorraine": ["08", "10", "51", "52", "54", "55", "57", "67", "88", "68"],
"Normandie": ["14", "27", "50", "61", "76"],
"Bretagne": ["22", "29", "35", "56"],
"Pays de la Loire": ["44", "49", "53", "72", "85"],
"Centre Val de Loire": ["18", "28", "36", "37", "41", "45"],
"Bourgogne Franche-Comté": ["21", "25", "39", "58", "70", "71", "89", "90"],
"Aquitaine Limousin Poitou-Charentes": ["16", "17", "19", "23", "24", "33", "40", "47", "64", "79", "86", "87"],
"Auvergne-Rhône-Alpes": ["01", "03", "07", "15", "26", "38", "42", "43", "63", "69", "73", "74"],
"Languedoc-Roussillon Midi Pyrénées": ["09", "11", "12", "30", "31", "32", "34", "46", "48", "65", "66", "81", "82"],
"Provence Alpes Côte d'Azur": ["04", "05", "06", "13", "83", "84"],
"Corse": ["20"],
}

In [8]:
def preprocess_text(text):
    text = " ".join(gensim.utils.simple_preprocess(text))
    return text

def strip_zipCode(zipCode):
    return str(zipCode)[0:2]

def get_region_by_stripped_zipCode(zipCode):
    region_index = None
    for idx, region in enumerate(region_names_to_zipCode_regions):
        if zipCode in region_names_to_zipCode_regions[region]:
            region_index = idx
    return region_index

def preprocess_zipCode(zipCode):
    zipCode = strip_zipCode(zipCode)
    return get_region_by_stripped_zipCode(zipCode)

Here we get the regions and answer text out of the dataframe, converting postal code to the region index

In [9]:
%%time

db = db_questions

texts = []
authorZipCode_regions = []
# question_numbers = []
i = 0
invalid_zipcodes = []

for row in range(db.shape[0]):
    if row % 10000 == 0: print("row", row, "/", db.shape[0])
    for col in range(db.shape[1]):
        if not pd.isnull(db.iat[row, col]):
            text = preprocess_text(db.iat[row, col])
            authorZipCode = db_raw.at[row, 'authorZipCode']
            authorZipCode_region = preprocess_zipCode(authorZipCode)
#             question_number = col
            
            if authorZipCode_region == None:
                invalid_zipcodes.append(strip_zipCode(authorZipCode))
                authorZipCode_region = -1

            texts.append(text)
            authorZipCode_regions.append(authorZipCode_region)
#             question_numbers.append(question_number)

invalid_zipcodes = set(invalid_zipcodes)
print("zipCodes that does not belong to any region:", invalid_zipcodes)

row 0 / 65758
row 10000 / 65758
row 20000 / 65758
row 30000 / 65758
row 40000 / 65758
row 50000 / 65758
row 60000 / 65758
zipCodes that does not belong to any region: {'3', '96', '5', '6', '97', '4', '99', '1', '9', '98', '7', '0'}
Wall time: 2min 29s


In [12]:
texts_backup = texts.copy()
authorZipCode_regions_backup = authorZipCode_regions.copy()

exclude zipcodes that were not included to any of the regions

In [14]:
for idx, zipCode in enumerate(authorZipCode_regions):
    if zipCode == -1:
        del texts[idx]
        del authorZipCode_regions[idx]

In [53]:
%%time
# strip_accents = 'ascii' almost has not changed the model
# limiting generated vocab size with max_features = ... decreases acc
vectorizer = TfidfVectorizer(min_df=1)
train_tfidf_texts = vectorizer.fit_transform(texts)

Wall time: 49.9 s


In [54]:
X = train_tfidf_texts
# X = train_tfidf_texts
# adding information about the question number increases accurcy very little
# X = hstack((train_tfidf_texts, np.array(question_numbers)[:,None]))
y = authorZipCode_regions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

### XGBoost

In [36]:
dtrain = xgb.DMatrix(X_train, label = y_train)

In [ ]:
param = {}
# param['silent'] = 1
# param['verbosity'] = 2
param['nthread'] = 4

param['max_depth'] = 6
num_boost_round = 10


bst = xgb.train(param, dtrain, num_boost_round = num_boost_round)

In [38]:
xgb_X_test = xgb.DMatrix(X_test)
xgb_y_test = xgb.DMatrix(y_test)

y_pred = bst.predict(xgb_X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Model accuracy: %.2f%%" % (accuracy * 100.0))
random_accuracy = 1 / len(set(authorZipCode_regions))
print("Random accuracy: %.2f%%" % (random_accuracy * 100.0))

Model accuracy: 5.66%
Random accuracy: 7.14%


### Naive Bayes

In [55]:
from sklearn.naive_bayes import MultinomialNB as NB

# fit_prior = False decreases acc by 0.1% which means that we have a slight imbalance between zipcodes
nb_model = NB().fit(X=X_train, y=y_train)

In [56]:
y_pred = nb_model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Model accuracy: %.2f%%" % (accuracy * 100.0))
random_accuracy = 1 / len(set(authorZipCode_regions))
print("Random accuracy: %.2f%%" % (random_accuracy * 100.0))

Model accuracy: 21.56%
Random accuracy: 7.14%


### Conclusion
As we can see, though doc2vec that can take the meaning of the sentence has performed poorly, TF-IDF and Naive Bayes gave pretty descent results  
Naive Bayes seems to be naive (huh) but can actually handle sparse data greatly because it is robust to the missing values in data (and we have almost all the data missing because the word frequence vector for each example is very sparse)

The code below allows to convert the texts = ['asdf', 'fdsa', ...] list with tags = [1, 0, 2, 1, 1, ..] to the texts and tags lists where the all the texts that correspond to the similar tag are joined together (it will be usefull in the following research)

In [17]:
# We don't need it here, actually

# thousand times slower
# def join_text_with_same_tag(texts, tags):
#     new_texts = []
#     new_tags = []
#     tag_to_text = {}
#     for tag in set(tags):
#         tag_to_text[tag] = ""
    
#     for idx, tag in enumerate(tags):
# #         if idx % (len(tags) // 10) == 0: print("%.0f%% done" % (idx / len(tags)))
#         print_progress_bar(idx, len(tags))
#         tag_to_text[tag] = " ".join((tag_to_text[tag], texts[idx]))
#     for tag in tag_to_text:
#         new_texts.append(tag_to_text[tag])
#         new_tags.append(tag)
#     return new_texts, new_tags

In [11]:
# We don't need it here, actually

# def join_text_with_same_tag(texts, tags):
#     new_texts = []
#     new_tags = []
#     tag_to_text = {}
#     for tag in set(tags):
#         tag_to_text[tag] = []
    
#     for idx, tag in enumerate(tags):
#         if idx % (len(tags) // 10) == 0: print("%.0f%% done" % (idx / len(tags) * 100))
#         tag_to_text[tag].append(texts[idx])
#     for tag in tag_to_text:
#         tag_to_text[tag] = " ".join(tag_to_text[tag])
#     for tag in tag_to_text:
#         new_texts.append(tag_to_text[tag])
#         new_tags.append(tag)
#     return new_texts, new_tags

In [14]:
# We don't need it here, actually

#%%time
#region_joined_texts, region_joined_tags = join_text_with_same_tag(texts, authorZipCode_regions)

0% done
10% done
20% done
30% done
40% done
50% done
60% done
70% done
80% done
90% done
100% done
CPU times: user 901 ms, sys: 72 ms, total: 973 ms
Wall time: 972 ms
